# Mini Project

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        # TODO BIAS = TRUE
        self.block = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=in_planes,
                                out_channels=planes,
                                kernel_size=3,
                                stride=stride,
                                padding=1,
                                bias=True),
                torch.nn.BatchNorm2d(planes),
                torch.nn.ReLU(inplace=True),
                torch.nn.Conv2d(in_channels=planes,
                                out_channels=planes,
                                kernel_size=3,
                                stride=1,
                                padding=1,
                                bias=True),
                torch.nn.BatchNorm2d(planes)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels=in_planes,
                          out_channels=self.expansion*planes,
                          kernel_size=1,
                          stride=stride,
                          bias=True),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        block = self.block(x)
        shortcut = self.shortcut(x)
        out = torch.nn.functional.relu(block+shortcut)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 256, num_blocks[3], stride=2)
        #self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        #self.linear = nn.Linear(512*block.expansion, num_classes)
        self.linear = nn.Linear(256*block.expansion, num_classes)
        
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [3,5, 2,1])

In [27]:
from torchsummary import summary

model = ResNet18()

summary(model.cuda(), (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,928
       BatchNorm2d-4           [-1, 64, 32, 32]             128
              ReLU-5           [-1, 64, 32, 32]               0
            Conv2d-6           [-1, 64, 32, 32]          36,928
       BatchNorm2d-7           [-1, 64, 32, 32]             128
        BasicBlock-8           [-1, 64, 32, 32]               0
            Conv2d-9           [-1, 64, 32, 32]          36,928
      BatchNorm2d-10           [-1, 64, 32, 32]             128
             ReLU-11           [-1, 64, 32, 32]               0
           Conv2d-12           [-1, 64, 32, 32]          36,928
      BatchNorm2d-13           [-1, 64, 32, 32]             128
       BasicBlock-14           [-1, 64,

## Train 

In [29]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
#import matplotlib.pyplot as plt

In [30]:
# Data
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [32]:
ROOT = '.data'
trainset = torchvision.datasets.CIFAR10(
    root=ROOT, train=True, download=True, transform=transform_train)
# trainloader = torch.utils.data.DataLoader(
#     trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root=ROOT , train=False, download=True, transform=transform_test)
# testloader = torch.utils.data.DataLoader(
#     testset, batch_size=100, shuffle=False, num_workers=2)
VALID_RATIO = 0.9

n_train_examples = int(len(trainset) * VALID_RATIO)
n_valid_examples = len(trainset) - n_train_examples
print(n_train_examples)
print(n_valid_examples)
generator=torch.Generator().manual_seed(42)

train_data, valid_data = torch.utils.data.random_split(trainset, 
                                           [n_train_examples, n_valid_examples], generator = generator)

0.2%

100.0%


Extracting .data/cifar-10-python.tar.gz to .data
Files already downloaded and verified
45000
5000


In [33]:
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=256, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=256, shuffle=False, num_workers=2)

In [38]:
#model 
net = ResNet18()
net = net.to('cuda')
device = 'cuda'

In [39]:
# define loss function, optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9,weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
save_loss = {'train':[], 'test':[]}
save_acc = {'train':[], 'test':[]}

In [40]:
# Training
import time
import os
import sys
import time
import math
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        print(batch_idx, len(trainloader), 'Train Loss: %.3f | Acc: %.3f%% (%d/%d)'% (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            print((batch_idx, len(testloader), 'Test Loss: %.3f | Acc: %.3f%% (%d/%d)'% (test_loss/(batch_idx+1), 100.*correct/total, correct, total)))



In [ ]:
for epoch in range(50):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 0
0 196 Train Loss: 1.768 | Acc: 36.328% (93/256)
1 196 Train Loss: 1.799 | Acc: 34.570% (177/512)
2 196 Train Loss: 1.786 | Acc: 33.073% (254/768)
3 196 Train Loss: 1.793 | Acc: 32.910% (337/1024)
4 196 Train Loss: 1.789 | Acc: 32.969% (422/1280)
5 196 Train Loss: 1.788 | Acc: 33.008% (507/1536)
6 196 Train Loss: 1.781 | Acc: 33.482% (600/1792)
7 196 Train Loss: 1.774 | Acc: 33.594% (688/2048)
8 196 Train Loss: 1.773 | Acc: 33.898% (781/2304)
9 196 Train Loss: 1.772 | Acc: 33.789% (865/2560)
10 196 Train Loss: 1.766 | Acc: 34.411% (969/2816)
11 196 Train Loss: 1.760 | Acc: 34.733% (1067/3072)
12 196 Train Loss: 1.764 | Acc: 34.495% (1148/3328)
13 196 Train Loss: 1.758 | Acc: 34.738% (1245/3584)
14 196 Train Loss: 1.762 | Acc: 34.609% (1329/3840)
15 196 Train Loss: 1.762 | Acc: 34.570% (1416/4096)
16 196 Train Loss: 1.760 | Acc: 34.467% (1500/4352)
17 196 Train Loss: 1.757 | Acc: 34.744% (1601/4608)
18 196 Train Loss: 1.754 | Acc: 34.581% (1682/4864)
19 196 Train Loss: 1.752 | 